In [1]:
!pip install xgboost

In [2]:
!pip install optuna

     |████████████████████████████████| 308 kB 4.1 MB/s 
     |████████████████████████████████| 81 kB 7.2 MB/s 
     |████████████████████████████████| 210 kB 50.9 MB/s 
     |████████████████████████████████| 75 kB 4.7 MB/s 
     |████████████████████████████████| 49 kB 6.3 MB/s 
     |████████████████████████████████| 150 kB 52.5 MB/s 
     |████████████████████████████████| 113 kB 52.1 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=0284c676c800fb7edbea27a05c530d599f9633d13886934943001b1c1383a59b
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [3]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

In [4]:
cd ./drive/MyDrive/미니_프로젝트/data

/content/drive/MyDrive/미니_프로젝트/data


# 모든 feature가 다 포함

In [5]:
train_fast = pd.read_csv('./train_fast_all.csv')
test_fast = pd.read_csv('./test_fast_all.csv')
train_slow = pd.read_csv('./train_slow_all.csv')
test_slow = pd.read_csv('./test_slow_all.csv')

In [6]:
train_fast.shape, test_fast.shape, train_slow.shape, test_slow.shape

((102316, 68), (25575, 68), (102294, 68), (25575, 68))

## 급속 충전기

In [7]:
X_fast_train = train_fast.drop(['fast_exist'], axis=1)
y_fast_train = train_fast.fast_exist

X_fast_test = test_fast.drop(['fast_exist'], axis=1)
y_fast_test = test_fast.fast_exist

### XGBoost

In [8]:
XGB = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1,
              n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1)

In [9]:
XGB.fit(X_fast_train, y_fast_train)

XGBClassifier(interaction_constraints='', learning_rate=0.300000012,
              max_depth=6, n_jobs=-1, num_parallel_tree=1, tree_method='exact',
              validate_parameters=1)

In [ ]:
def XGB_objective(trial):
    base_score = 0.5
    booster = trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart'])
    colsample_bylevel = trial.suggest_float('colsample_bylevel', 0, 1)
    colsample_bynode = trial.suggest_float('colsample_bynode', 0, 1)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0, 1)
    gamma = trial.suggest_float('gamma', 0, 10)
    importance_type = trial.suggest_categorical('importance_type', ['weight', 'gain', 'cover', 'total_gain', 'total_cover'])
    learning_rate = trial.suggest_float('learning_rate', 0, 1)
    max_delta_step = trial.suggest_float('max_delta_step', 0, 10)
    max_depth = trial.suggest_int('max_depth', 1, 10)
    min_child_weight = trial.suggest_float('min_child_weight', 0, 10)
    n_estimators =  trial.suggest_int('n_estimators', 100, 500)
    n_jobs = -1
    num_parallel_tree = trial.suggest_int('num_parallel_tree', 1,10)
    random_state = 42
    reg_alpha=0
    reg_lambda=1
    scale_pos_weight=1
    subsample=1
    tree_method='gpu_hist'
    validate_parameters=1
    
   
    model = XGBClassifier(base_score=base_score, booster=booster, colsample_bylevel=colsample_bylevel, colsample_bytree=colsample_bytree, colsample_bynode=colsample_bynode,
                          gamma=gamma, importance_type=importance_type, learning_rate=learning_rate, max_delta_step=max_delta_step, max_depth=max_depth, min_child_weight=min_child_weight,
                          n_estimators=n_estimators, n_jobs=n_jobs, num_parallel_tree=num_parallel_tree, random_state=random_state, reg_alpha=reg_alpha, reg_lambda=reg_lambda,
                          scale_pos_weight=scale_pos_weight, subsample=subsample, tree_method=tree_method, validate_parameters=validate_parameters
                          )

    
    model.fit(X_fast_train, y_fast_train)
    y_fast_pred = XGB.predict(X_fast_test)
    f1 = f1_score(y_fast_test, y_fast_pred)

    return f1
    
#Execute optuna and set hyperparameters
XGB_study = optuna.create_study(direction='maximize')
XGB_study.optimize(XGB_objective, n_trials=10)

#Create an instance with tuned hyperparameters
optimized_XGB = XGBClassifier(base_score=XGB_study.best_params['base_score'], booster=XGB_study.best_params['booster'], colsample_bylevel=XGB_study.best_params['colsample_bylevel'], colsample_bytree=XGB_study.best_params['colsample_bytree'], 
                              colsample_bynode=XGB_study.best_params['colsample_bynode'], gamma=XGB_study.best_params['gamma'], importance_type=XGB_study.best_params['importance_type'], learning_rate=XGB_study.best_params['learning_rate'], 
                              max_delta_step=XGB_study.best_params['max_delta_step'], max_depth=XGB_study.best_params['max_depth'], min_child_weight=XGB_study.best_params['min_child_weight'], n_estimators=XGB_study.best_params['n_estimators'], 
                              n_jobs=XGB_study.best_params['n_jobs'], num_parallel_tree=XGB_study.best_params['num_parallel_tree'], random_state=XGB_study.best_params['random_state'], reg_alpha=XGB_study.best_params['reg_alpha'], 
                              reg_lambda=XGB_study.best_params['reg_lambda'], scale_pos_weight=XGB_study.best_params['scale_pos_weight'], subsample=XGB_study.best_params['subsample'], tree_method=XGB_study.best_params['tree_method'], 
                              validate_parameters=XGB_study.best_params['validate_parameters']                              
                              )

[I 2022-02-23 05:20:11,724] A new study created in memory with name: no-name-c41f8de1-de60-4896-a148-a025cf0d12d6
[I 2022-02-23 05:26:29,139] Trial 0 finished with value: 0.72 and parameters: {'booster': 'dart', 'colsample_bylevel': 0.5059127529825126, 'colsample_bynode': 0.28284653552690986, 'colsample_bytree': 0.818329154113998, 'gamma': 2.038004375785465, 'importance_type': 'cover', 'learning_rate': 0.6578520384062533, 'max_delta_step': 1.4102894706753621, 'max_depth': 10, 'min_child_weight': 4.365294482716741, 'n_estimators': 297, 'num_parallel_tree': 7}. Best is trial 0 with value: 0.72.
[I 2022-02-23 05:31:12,509] Trial 1 finished with value: 0.72 and parameters: {'booster': 'dart', 'colsample_bylevel': 0.6876849695235908, 'colsample_bynode': 0.2648135844885716, 'colsample_bytree': 0.2675236821277176, 'gamma': 0.7949200953077085, 'importance_type': 'weight', 'learning_rate': 0.45025941153907456, 'max_delta_step': 4.965728224682802, 'max_depth': 7, 'min_child_weight': 6.8992614100

In [10]:
y_fast_prob = XGB.predict_proba(X_fast_test)

In [11]:
for threshold in range(1,10):
    threshold = round(threshold*0.1, 1)
    y_fast_pred = np.where(y_fast_prob>=threshold, 1, 0)
    print(f'{threshold} :{f1_score(y_fast_test, y_fast_pred[:,1])}')

0.1 :0.6176470588235294
0.2 :0.7241379310344828
0.3 :0.7169811320754718
0.4 :0.7307692307692308
0.5 :0.72
0.6 :0.7346938775510204
0.7 :0.7083333333333334
0.8 :0.6808510638297872
0.9 :0.6666666666666665


* **오버피팅!!!!!!!!!**

In [12]:
y_fast_pred_train = XGB.predict(X_fast_train)
f1_score(y_fast_train, y_fast_pred_train)

1.0

## 완속 충전기

In [ ]:
X_slow_train = train_slow.drop(['slow_exist'], axis=1)
y_slow_train = train_slow.slow_exist

X_slow_test = test_slow.drop(['slow_exist'], axis=1)
y_slow_test = test_slow.slow_exist

### XGBoost

In [ ]:
XGB_slow = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1,
              n_estimators=100, n_jobs=-1, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1)

In [ ]:
XGB_slow.fit(X_slow_train, y_slow_train)

XGBClassifier(interaction_constraints='', learning_rate=0.300000012,
              max_depth=6, n_jobs=-1, num_parallel_tree=1, tree_method='exact',
              validate_parameters=1)

In [ ]:
y_slow_prob = XGB_slow.predict_proba(X_slow_test)

In [ ]:
for threshold in range(1,10):
    threshold = round(threshold*0.1, 1)
    y_slow_pred = np.where(y_slow_prob>=threshold, 1, 0)
    print(f'{threshold} :{f1_score(y_slow_test, y_slow_pred[:,1])}')

0.1 :0.625
0.2 :0.6666666666666667
0.3 :0.7073170731707317
0.4 :0.725
0.5 :0.7179487179487181
0.6 :0.7397260273972602
0.7 :0.75
0.8 :0.7428571428571429
0.9 :0.7076923076923077
